<div class="alert alert-block alert-info">
<h2>Import packages:</h2>
</div>

In [1]:
import pandas as pd
import datetime as dt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'svg'

---

<div class="alert alert-block alert-info">
<h2>Import Data & Explore:</h2>
</div>

In [ ]:
retail_df = pd.read_excel('/Users/walter/Desktop/REPOS/analytics-stories/Cohort-Analysis-Retail/data/Online_Retail.xlsx')
retail_df

### Get the percent of null datapoints per column:

In [ ]:
pd.DataFrame(retail_df.isnull().mean().round(4)*100, columns=['Percent Null'])

<div class="alert alert-block alert-warning">
<b>Note:</b> Approximately 25% of the data is missing for the custome IDs. We will have to drop these.
</div>

In [ ]:
retail_df = retail_df.dropna(subset=['CustomerID'])
pd.DataFrame(retail_df.isnull().mean().round(4)*100, columns=['Percent Null'])

<div class="alert alert-block alert-success">
<b>Note:</b> now we have no missing records.
</div>

---

<div class="alert alert-block alert-info">
<h2>Prepare Data for Cohort Analysis:</h2>
</div>

### Fist we build a function to handle the date formats:
Convert the date column into a `datetime object` and then extract month and year.

In [ ]:
def date_handler(df, col):
    # Operate on a copy to avoid SettingWithCopyWarning:
    transformed_df = df.copy()
    
    # Convert the date column to datetime and create new columns:
    transformed_df[col] = pd.to_datetime(transformed_df[col])
    
    # Group by CustomerID and get the first purchase date:
    first_purchase_df = transformed_df.groupby('CustomerID')[col].min().reset_index()
    first_purchase_df.rename(columns={col: 'FirstPurchaseDate'}, inplace=True)
    
    # Merge to get the 'FirstPurchase' column in the main dataframe:
    transformed_df = transformed_df.merge(first_purchase_df, how='left', on='CustomerID')

    # Ensure that the 'FirstPurchaseDate' is a datetime object as well:
    transformed_df['FirstPurchaseDate'] = pd.to_datetime(transformed_df['FirstPurchaseDate'])

    # Get the number of months the user is active by subtracting the invoice date from the 1st purchase date.
    # Ensure 'FirstPurchaseDate' is earlier than 'InvoiceDate' before subtracting to get 'MonthsActive':
    transformed_df['MonthsActive'] = (
        (transformed_df['InvoiceDate'].dt.year - transformed_df['FirstPurchaseDate'].dt.year) * 12
        + (transformed_df['InvoiceDate'].dt.month - transformed_df['FirstPurchaseDate'].dt.month) + 1
    )

    # # If the day of 'InvoiceDate' is earlier than the day of 'FirstPurchaseDate', subtract one month:
    # mask = transformed_df['InvoiceDate'].dt.day < transformed_df['FirstPurchaseDate'].dt.day
    # transformed_df.loc[mask, 'MonthsActive'] -= 1

    # # Ensure 'MonthsActive' is not negative (do zero if so)
    # transformed_df['MonthsActive'] = transformed_df['MonthsActive'].clip(lower=0)

    # Create Cohort Month column:
    transformed_df['CohortMonth'] = (transformed_df['FirstPurchaseDate'].dt.year.astype(str)
                                    + '-' + transformed_df['FirstPurchaseDate'].dt.month.astype(str).str.zfill(2))

    return transformed_df


In [ ]:
retail_df = date_handler(retail_df, 'InvoiceDate')
retail_df

---

<div class="alert alert-block alert-info">
<h2>Create Cohort table:</h2>
</div>

### First group by CohortMonth and #of months active to get the count of unique customers for that cohort month and active month combination:

In [ ]:
cohort_df = retail_df.groupby(['CohortMonth', 'MonthsActive'])['CustomerID'].nunique().reset_index().rename(columns={'CustomerID': 'UniqueCustomers'})
cohort_df

### Create a cohort table by pivoting on the grouped dataframe from the previous step:

In [ ]:
cohort_table = pd.pivot_table(data=cohort_df, columns='MonthsActive', index='CohortMonth', values='UniqueCustomers')
cohort_table.fillna('')

### We can get the percent of each cohort that remains per active month:

In [ ]:
cohort_table_percent = (cohort_table.divide(cohort_table.iloc[:, 0], axis=0)*100).round()
cohort_table_percent.fillna('')

---

<div class="alert alert-block alert-info">
<h2>Visualize Cohort table:</h2>
</div>

### Create a function to plot the cohort table:

In [ ]:
def plot_cohort_table(df, heat_legend, colormap):
    # Convert the percentage values to a formatted string
    text_values = df.map('{:.2f}%'.format)
    
    # Create the heatmap with Plotly Express and display cell values
    fig = px.imshow(
        df,
        labels=dict(x="Months Active", y="Cohort Month", color=heat_legend),
        x=df.columns,
        y=df.index,
        text_auto=True,  # You can set this to False if you're providing z_text
        # If you want specific formatting or to use the text_values from above, comment out text_auto and use:
        # z_text=text_values.values,
        color_continuous_scale=colormap
    )
    
    # Update the layout
    fig.update_layout(
        title='Cohort Analysis',
        xaxis=dict(side="bottom", tickvals=df.columns),
        yaxis=dict(tickvals=df.index),
        height=800
    )
    
    # Show the figure
    return fig.show(width=1600)
    

### Plot:

In [ ]:
plot_cohort_table(cohort_table_percent, 'Percent Active', 'Reds')

---

<div class="alert alert-block alert-info">
<h2>Let's create a similar table but we are going to look at the cohort LTV this time:</h2>
</div>

### Begin with the preprocessed retail_df and go from there:

In [ ]:
retail_df

### Calculate the order total from unit price and quantity:

In [ ]:
retail_df['OrderTotal'] = retail_df['UnitPrice']*retail_df['Quantity']
retail_df

### First group by CohortMonth and #of months active to get the sum of OrderTotal for that cohort month and active month combination:

In [ ]:
cohort_ltv_df = retail_df.groupby(['CohortMonth', 'MonthsActive']).agg(
    TotalOrderValue=('OrderTotal', 'sum'),
    UniqueCustomers=('CustomerID', 'nunique')).reset_index()
cohort_ltv_df

### Get the average order value per customer for each of the cohort months and months active:

In [ ]:
cohort_ltv_df['AvgCohortOrderValue'] = (cohort_ltv_df['TotalOrderValue']/cohort_ltv_df['UniqueCustomers']).round(2)
cohort_ltv_df

<div class="alert alert-block alert-info">
<h2>Build Cohort LTV table:</h2>
</div>

### Create a pivot table:

In [ ]:
cohort_ltv_table = pd.pivot_table(data=cohort_ltv_df, columns='MonthsActive', index='CohortMonth', values='AvgCohortOrderValue')
cohort_ltv_table.fillna('')

### Plot to see avg. order for each cohort and month active combination:

In [ ]:
plot_cohort_table(cohort_ltv_table, 'Cohort Avg. Order', 'Reds')

### Get the cumulative sum for each cohort month and months active:

In [ ]:
cohort_ltv_cumulative = cohort_ltv_table.cumsum(axis=1)
cohort_ltv_cumulative.fillna('')

### Plot cumulative sum table to get the LTV of each cohort and months active combination:

In [ ]:
plot_cohort_table(cohort_ltv_cumulative, 'Cohort LTV', 'Reds')